In [28]:
##### Package
library("readstata13")
library(plyr)
library(dplyr)
library(tidyr)
library(Gmisc) # Transition Games

### Directories
path_work <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/"
graph_path <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/05_Paper/Graph/"
save_data <-  "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/05_Analysis/"
path_who <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/05_Analysis/inputs/"



# Functions

In [29]:
get_area_crop <- function(df) {

    #### Get Classification 
    crop_land_14 <- df %>% 
                filter(!is.na(area_crop)) %>%
        mutate(
      ##### First Classification - With Bananas
      trees_a=ifelse((cropID %in% c(710,700,750,760,770,741,742,744)),"Fruit Trees",
                     ifelse(cropID %in% c(860,820,830,810,811,812),"Tree Cash Crops",
                    ifelse(cropID %in% c(880,970,990,950,960),"Trees for Timber/Fuelwood","Other Crops"))),
      ##### Second Classification - With Bananas
      trees_b=ifelse((cropID %in% c(710,700,750,760,770)),"Fruit Trees",
                     ifelse(cropID %in% c(860,820,830,810,811,812),"Tree Cash Crops",
                            ifelse(cropID %in% c(880,970,990,950,960),"Trees for Timber/Fuelwood","Other Crops"))))
  

  #### Building Aggregation for the first variable
  crop_land_14_def_1 <- crop_land_14 %>% group_by(hhid,parcel_id,trees_a) %>% 
    summarise(area_crop=sum(area_crop,na.rm=TRUE)) %>% 
    ### ---> Spread the data and rename
    spread(trees_a,area_crop,fill=0) %>% 
    rename(fruit=`Fruit Trees`,other_crop=`Other Crops`,cash=`Tree Cash Crops`) %>%
    ### ---> Rename all varibles
    setNames(c(names(.)[1], paste0("def1_",names(.)[-1]))) %>% rename(parcel_id=def1_parcel_id)
  
  #### Building Aggregation for the Second variable
  crop_land_14_def_2 <- crop_land_14 %>% group_by(hhid,parcel_id,trees_b) %>% 
    summarise(area_crop=sum(area_crop,na.rm=TRUE)) %>% 
    ### ---> Spread the data and rename
    spread(trees_b,area_crop,fill=0) %>% 
    rename(fruit=`Fruit Trees`,other_crop=`Other Crops`,cash=`Tree Cash Crops`) %>%
    ### ---> Rename all varibles
    setNames(c(names(.)[1], paste0("def2_",names(.)[-1]))) %>% 
    rename(parcel_id=def2_parcel_id) %>% mutate(parce_a_es=(def2_fruit+def2_other_crop+def2_cash))
  
  #### Merge information
  
  crop_a_14 <- crop_land_14_def_1 %>% inner_join(crop_land_14_def_2) %>% select(def1_fruit,parce_a_es,starts_with("def2_")) %>% 
    ### Make the aggregations
    transmute(treeonfarm_w_b=(def1_fruit+def2_cash),treesonfarm_wo_b=(def2_fruit+def2_cash),fruit_w_b=def1_fruit,fruit_wo_b=def2_fruit,cash=def2_cash,land_size_est=parce_a_es)

    ##3
    return(crop_a_14)
}

# Wave 2005-06

## Area Per Crop

In [30]:
############################################
  ###############------------- Area per crop
  #############################################
  crop_land_05 <- read.dta13(paste0(path_work,"00_RawData/2005-06/Agriculture/AGSEC4A.dta"))
  
  crop_land_05 <- crop_land_05 %>% transmute(hhid=HHID,parcel_id=a4aq1,plot_id=a4aq2,cropID=a4aq5b,plot_size=a4aq3,share_crop=(a4aq6/100)) %>% filter(!is.na(share_crop)) %>% transmute(hhid,parcel_id,plot_id,cropID,area_crop=plot_size*share_crop) %>%
    mutate(
    ##### First Classification - With Bananas
   trees_a=ifelse((cropID %in% c(710,700,750,760,770,741,742,744)),"Fruit Trees",
ifelse(cropID %in% c(860,820,830,810,811,812),"Tree Cash Crops",
ifelse(cropID %in% c(880,970,990,950,960),"Trees for Timber/Fuelwood","Other Crops"))),
  ##### Second Classification - With Bananas
trees_b=ifelse((cropID %in% c(710,700,750,760,770)),"Fruit Trees",
ifelse(cropID %in% c(860,820,830,810,811,812),"Tree Cash Crops",
ifelse(cropID %in% c(880,970,990,950,960),"Trees for Timber/Fuelwood","Other Crops"))))
  #### Building Aggregation for the first variable
  crop_land_05_def_1 <- crop_land_05 %>% group_by(hhid,parcel_id,trees_a) %>% summarise(area_crop=sum(area_crop,na.rm=TRUE)) %>% 
    ### ---> Spread the data and rename
    spread(trees_a,area_crop,fill=0) %>% 
    rename(fruit=`Fruit Trees`,other_crop=`Other Crops`,cash=`Tree Cash Crops`,timber=`Trees for Timber/Fuelwood`) %>%
    ### ---> Rename all varibles
    setNames(c(names(.)[1], paste0("def1_",names(.)[-1]))) %>% rename(parcel_id=def1_parcel_id)
  
  #### Building Aggregation for the Second variable
  crop_land_05_def_2 <- crop_land_05 %>% group_by(hhid,parcel_id,trees_b) %>% summarise(area_crop=sum(area_crop,na.rm=TRUE)) %>% 
    ### ---> Spread the data and rename
    spread(trees_b,area_crop,fill=0) %>% 
    rename(fruit=`Fruit Trees`,other_crop=`Other Crops`,cash=`Tree Cash Crops`,timber=`Trees for Timber/Fuelwood`) %>%
    ### ---> Rename all varibles
    setNames(c(names(.)[1], paste0("def2_",names(.)[-1]))) %>% rename(parcel_id=def2_parcel_id) %>% mutate(parce_a_es=(def2_fruit+def2_other_crop+def2_timber+def2_cash))
  
  #### Merge information
  
  crop_a_05 <- crop_land_05_def_1 %>% inner_join(crop_land_05_def_2) %>% select(def1_fruit,parce_a_es,starts_with("def2_")) %>% 
    ### Make the aggregations
    transmute(treeonfarm_w_b=(def1_fruit+def2_timber+def2_cash),treesonfarm_wo_b=(def2_fruit+def2_timber+def2_cash),fruit_w_b=def1_fruit,fruit_wo_b=def2_fruit,cash=def2_cash,timber=def2_timber,land_size_est=parce_a_es)

Joining, by = c("hhid", "parcel_id")
Adding missing grouping variables: `hhid`, `parcel_id`
Adding missing grouping variables: `hhid`, `parcel_id`


## Land Size

In [31]:
  #############################################
  ###############------------- Land Size
  #############################################
  
  #####---- Land Size (Ownd)
  land_05_a <- read.dta13(paste0(path_work,"00_RawData/2005-06/Agriculture/AGSEC2A.dta"))
  
  land_05_a <- land_05_a %>% transmute(hhid=HHID,parcel_id=a2aq2,land_size=ifelse(is.na(a2aq4),a2aq5,a2aq4)) 
  
  #####---- Land Size (Use Rights)
  land_05_b <- read.dta13(paste0(path_work,"00_RawData/2005-06/Agriculture/AGSEC2B.dta"))
  
  land_05_b <- land_05_b %>% transmute(hhid=HHID,parcel_id=a2bq2,land_size=ifelse(is.na(a2bq4),a2bq5,a2bq4)) 
  
  #### Append Data Set
  land_05 <- rbind(land_05_a,land_05_b)
  

Warning message in read.dta13(paste0(path_work, "00_RawData/2005-06/Agriculture/AGSEC2A.dta")):
“
  a2aq13b:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Warning message in read.dta13(paste0(path_work, "00_RawData/2005-06/Agriculture/AGSEC2B.dta")):
“
  a2bq15a:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”

## Final Merge

In [32]:
  #############################################
  ###############------------- Final Merge
  #############################################

  agri_06 <- crop_a_05 %>% left_join(land_05) %>% filter(!is.na(land_size)) %>%
    mutate(land_size=ifelse(land_size_est>land_size,land_size_est,land_size)) %>%
    #### Gen the aggregation by hiig
    group_by(hhid) %>% summarise_all(sum) %>% select(-parcel_id) %>%
    #### Build the shares
    transmute(hhid=hhid,sh_tof_w_b=treeonfarm_w_b/land_size,sh_tof_wo_b=treesonfarm_wo_b/land_size,sh_fruit_w_b=fruit_w_b/land_size,sh_fruit_wo_b=fruit_wo_b/land_size,sh_cash=cash/land_size,sh_timber=timber/land_size,land_size=land_size)
  

Joining, by = c("hhid", "parcel_id")


# Wave 2009-10

## Area Crop

In [33]:
crop10 <- read.dta13(paste0(path_work,"00_RawData/2009-10/Agriculture/AGSEC4A.dta")) %>% 
        ### Identify Variables
        transmute(hhid=HHID,parcel_id=a4aq2,plot_id=a4aq4,cropID=a4aq6,area_crop=a4aq8) 

crop_a_10 <- get_area_crop(crop10)


Joining, by = c("hhid", "parcel_id")
Adding missing grouping variables: `hhid`, `parcel_id`
Adding missing grouping variables: `hhid`, `parcel_id`


##  Land Size

In [34]:
  
#####---- Land Size (Ownd)
land_a <- read.dta13(paste0(path_work,"00_RawData/2009-10/Agriculture/AGSEC2A.dta"))
land_a <- land_a %>% transmute(hhid=HHID,parcel_id=as.integer(a2aq2),land_size=ifelse(is.na(a2aq4),a2aq5,a2aq4)) %>% 
        group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE))

#####---- Land Size (Use Rights)
land_b <- read.dta13(paste0(path_work,"00_RawData/2009-10/Agriculture/AGSEC2B.dta"))
land_b <- land_b %>% transmute(hhid=HHID,parcel_id=as.integer(a2bq2),land_size=ifelse(is.na(a2bq4),a2bq5,a2bq4)) %>% 
    group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE)) 
 
#### Append Data Set
land_10 <- rbind(land_a,land_b)
  

Warning message in read.dta13(paste0(path_work, "00_RawData/2009-10/Agriculture/AGSEC2A.dta")):
“
  a2aq6:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Warning message in read.dta13(paste0(path_work, "00_RawData/2009-10/Agriculture/AGSEC2B.dta")):
“
  a2bq6:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Warning message in read.dta13(paste0(path_work, "00_RawData/2009-10/Agriculture/AGSEC2B.dta")):
“
  a2bq8:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Warning message in read.dta13(paste0(path_work, "00_RawData/2009-10/Agriculture/AGSEC2B.dta")):
“
  a2bq11:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”

## Merge 

In [35]:
agri_10 <- crop_a_10 %>% left_join(land_10) %>% filter(!is.na(land_own)) %>%
    mutate(land_size=ifelse(land_size_est>land_own,land_size_est,land_own)) %>%
    #### Gen the aggregation by hiig
    group_by(hhid) %>% summarise_all(sum) %>% select(-parcel_id) %>%
    #### Build the shares
    transmute(hhid=hhid,sh_tof_w_b=treeonfarm_w_b/land_size,sh_tof_wo_b=treesonfarm_wo_b/land_size,sh_fruit_w_b=fruit_w_b/land_size,sh_fruit_wo_b=fruit_wo_b/land_size,sh_cash=cash/land_size,land_size=land_size)
  

Joining, by = c("hhid", "parcel_id")


# Wave 2010-11

## Area Per Crop

In [36]:
crop11 <- read.dta13(paste0(path_work,"00_RawData/2010-11/Agriculture/AGSEC4A.dta")) %>% 
        ### Identify Variables
        transmute(hhid=HHID,parcel_id=prcid,plot_id=pltid,cropID=cropID,area_crop=a4aq8) 

crop_a_11 <- get_area_crop(crop11)


Warning message in read.dta13(paste0(path_work, "00_RawData/2010-11/Agriculture/AGSEC4A.dta")):
“
  prcid:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Warning message in read.dta13(paste0(path_work, "00_RawData/2010-11/Agriculture/AGSEC4A.dta")):
“
  cropID:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Joining, by = c("hhid", "parcel_id")
Adding missing grouping variables: `hhid`, `parcel_id`
Adding missing grouping variables: `hhid`, `parcel_id`


## Land Size

In [37]:
  
#####---- Land Size (Ownd)
land_a <- read.dta13(paste0(path_work,"00_RawData/2010-11/Agriculture/AGSEC2A.dta"))
land_a <- land_a %>% transmute(hhid=HHID,parcel_id=as.integer(prcid),land_size=ifelse(is.na(a2aq4),a2aq5,a2aq4)) %>% 
        group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE))

#####---- Land Size (Use Rights)
land_b <- read.dta13(paste0(path_work,"00_RawData/2010-11/Agriculture/AGSEC2B.dta"))
land_b <- land_b %>% transmute(hhid=HHID,parcel_id=as.integer(prcid),land_size=ifelse(is.na(a2bq4),a2bq5,a2bq4)) %>% 
    group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE)) 
 
#### Append Data Set
land_11 <- rbind(land_a,land_b)
  

Warning message in read.dta13(paste0(path_work, "00_RawData/2010-11/Agriculture/AGSEC2B.dta")):
“
  prcid:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”

## Final Merge

In [38]:
agri_11 <- crop_a_11 %>% left_join(land_11) %>% filter(!is.na(land_own)) %>%
    mutate(land_size=ifelse(land_size_est>land_own,land_size_est,land_own)) %>%
    #### Gen the aggregation by hiig
    group_by(hhid) %>% summarise_all(sum) %>% select(-parcel_id) %>%
    #### Build the shares
    transmute(hhid=hhid,sh_tof_w_b=treeonfarm_w_b/land_size,sh_tof_wo_b=treesonfarm_wo_b/land_size,sh_fruit_w_b=fruit_w_b/land_size,sh_fruit_wo_b=fruit_wo_b/land_size,sh_cash=cash/land_size,land_size=land_size)
  

Joining, by = c("hhid", "parcel_id")


# Wave 2011-12

## Area per Crop

In [39]:
crop12 <- read.dta13(paste0(path_work,"00_RawData/2011-12/Agriculture/AGSEC4A.dta")) %>% 
        ### Identify Variables
        transmute(hhid=HHID,parcel_id=parcelID,plot_id=plotID,cropID=cropID,area_crop=a4aq7) 

crop_a_12 <- get_area_crop(crop12)

Warning message in read.dta13(paste0(path_work, "00_RawData/2011-12/Agriculture/AGSEC4A.dta")):
“
  cropID:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Joining, by = c("hhid", "parcel_id")
Adding missing grouping variables: `hhid`, `parcel_id`
Adding missing grouping variables: `hhid`, `parcel_id`


## Land Size

In [40]:
#####---- Land Size (Ownd)
land_a <- read.dta13(paste0(path_work,"00_RawData/2011-12/Agriculture/AGSEC2A.dta"))
land_a <- land_a %>% transmute(hhid=HHID,parcel_id=as.integer(parcelID),land_size=ifelse(is.na(a2aq4),a2aq5,a2aq4)) %>% 
        group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE))

#####---- Land Size (Use Rights)
land_b <- read.dta13(paste0(path_work,"00_RawData/2011-12/Agriculture/AGSEC2B.dta"))
land_b <- land_b %>% transmute(hhid=HHID,parcel_id=as.integer(parcelID),land_size=ifelse(is.na(a2bq4),a2bq5,a2bq4)) %>% 
    group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE)) 
 
#### Append Data Set
land_12 <- rbind(land_a,land_b)

## Merge

In [41]:
agri_12 <- crop_a_12 %>% left_join(land_12) %>% filter(!is.na(land_own)) %>%
    mutate(land_size=ifelse(land_size_est>land_own,land_size_est,land_own)) %>%
    #### Gen the aggregation by hiig
    group_by(hhid) %>% summarise_all(sum) %>% select(-parcel_id) %>%
    #### Build the shares
    transmute(hhid=hhid,sh_tof_w_b=treeonfarm_w_b/land_size,sh_tof_wo_b=treesonfarm_wo_b/land_size,sh_fruit_w_b=fruit_w_b/land_size,sh_fruit_wo_b=fruit_wo_b/land_size,sh_cash=cash/land_size,land_size=land_size)
  

Joining, by = c("hhid", "parcel_id")


# Wave 2013-14

## Area per Crop

In [42]:
crop14 <- read.dta13(paste0(path_work,"00_RawData/2013-14/AGSEC4A.dta")) %>% 
        ### Identify Variables
        transmute(hhid=hh,parcel_id=parcelID,plot_id=plotID,cropID=cropID,area_crop=a4aq7,year_planted=a4aq9_2) 

crop_a_14 <- get_area_crop(crop12)


Warning message in read.dta13(paste0(path_work, "00_RawData/2013-14/AGSEC4A.dta")):
“
  cropID:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Warning message in read.dta13(paste0(path_work, "00_RawData/2013-14/AGSEC4A.dta")):
“
  a4aq12b:
  Missing factor labels - no labels assigned.
  Set option generate.factors=T to generate labels.”Joining, by = c("hhid", "parcel_id")
Adding missing grouping variables: `hhid`, `parcel_id`
Adding missing grouping variables: `hhid`, `parcel_id`


## Land Size

In [43]:
#####---- Land Size (Ownd)
land_a <- read.dta13(paste0(path_work,"00_RawData/2011-12/Agriculture/AGSEC2A.dta"))
land_a <- land_a %>% transmute(hhid=HHID,parcel_id=as.integer(parcelID),land_size=ifelse(is.na(a2aq4),a2aq5,a2aq4)) %>% 
        group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE))

#####---- Land Size (Use Rights)
land_b <- read.dta13(paste0(path_work,"00_RawData/2011-12/Agriculture/AGSEC2B.dta"))
land_b <- land_b %>% transmute(hhid=HHID,parcel_id=as.integer(parcelID),land_size=ifelse(is.na(a2bq4),a2bq5,a2bq4)) %>% 
    group_by(hhid,parcel_id) %>% summarise(land_own=sum(land_size,na.rm=TRUE)) 
 
#### Append Data Set
land_12 <- rbind(land_a,land_b)

## Merge 

In [44]:
agri_14 <- crop_a_12 %>% left_join(land_12) %>% filter(!is.na(land_own)) %>%
    mutate(land_size=ifelse(land_size_est>land_own,land_size_est,land_own)) %>%
    #### Gen the aggregation by hiig
    group_by(hhid) %>% summarise_all(sum) %>% select(-parcel_id) %>%
    #### Build the shares
    transmute(hhid=hhid,sh_tof_w_b=treeonfarm_w_b/land_size,sh_tof_wo_b=treesonfarm_wo_b/land_size,sh_fruit_w_b=fruit_w_b/land_size,sh_fruit_wo_b=fruit_wo_b/land_size,sh_cash=cash/land_size,land_size=land_size)
  

Joining, by = c("hhid", "parcel_id")


# Final - Save Data Sets

In [45]:
agri_06 <- agri_06 %>% mutate(HHID=as.character(hhid))
agri_10 <- agri_10 %>% mutate(HHID=as.character(hhid))
agri_11 <- agri_11 %>% mutate(HHID=as.character(hhid))
agri_12 <- agri_12 %>% mutate(HHID=as.character(hhid))
agri_14 <- agri_14 %>% mutate(HHID=as.character(hhid))

In [46]:
save(agri_06,agri_10,agri_11,agri_12,agri_14,file=paste0(save_data,"03_agr_controls_June2017.Rda"))